
**Homework #1. linear regression model**

###이번 실습의 학습 목표 입니다.
1. 주어진 데이터를 불러오는 과정 익히기.
2. train loader & test loader 만들기.
3. model에 들어갈 input data preprocessing 해보기.
4. 간단한 linear regression model 구현해보기.
5. pytorch에 구현되어있는 loss function, optimizer 사용해보기.
6. 학습과정에서의 이해를 돕기 위해 기능들이 수행하는 역할에 대해 공부하기.

 + input값 채우기.
7.  Linear regression이 왜 Neural network인지 해당 과제를 예시로 들어 설명하기.

In [1]:
import torch
from torch import nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import dataloader

import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import os

import matplotlib.pyplot as plt

from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor

In [2]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

print('Current cuda device is', device)

Current cuda device is cpu


* 주어진 데이터를 불러오는 과정 익히기.

In [3]:
training_data=MNIST(root="./", train=True, download=True, transform=ToTensor())
test_data=MNIST(root="./", train=False, download=True, transform=ToTensor())

print("training data len : ",len(training_data))
print("test data len : ",len(test_data))
print(training_data)

# data image 9장 불러오기 - 'import matplotlib.pyplot as plt' 사용


training data len :  60000
test data len :  10000
Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train
    StandardTransform
Transform: ToTensor()


* train loader & test loader 만들기.

In [ ]:
# train loader & test loader 만들기 (batch size : 32/ train : shuffle=True, test : shuffle=False)
# 데이터셋을 DataLoader에 넣어 Train Loader와 Test Loader를 생성합니다.
from torch.utils.data import DataLoader
batch_size = 32  # 미니배치 크기를 32로 설정합니다.
shuffle_train = True  # 학습 데이터(shuffle_train)은 데이터를 섞음
shuffle_test = False  # 테스트 데이터(shuffle_test)는 섞지 않음.

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=shuffle_train)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=shuffle_test)

* input data preprocessing 해보기.

In [ ]:
# x_values = [i for i in range(11)]
# x_train = () # numpy를 사용하여 input data를 배열 형태로 만들기(type=np.float32)
# x_train = x_train.reshape(-1, 1)

# y_values = [2*i + 1 for i in x_values]
# y_train = () # numpy를 사용하여 input data를 배열 형태로 만들기(type=np.float32)
# y_train = y_train.reshape(-1, 1)
import numpy as np

x_values = np.array([i for i in range(11)], dtype=np.float32)
x_train = x_values.reshape(-1, 1)

y_values = np.array([2*i + 1 for i in x_values], dtype=np.float32)
y_train = y_values.reshape(-1, 1)

* 간단한 linear regression model 구현해보기.

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self, inputSize, outputSize):
        super(LinearRegression, self).__init__()
        # 선형 레이어 초기화: 입력 차원 -> 출력 차원
        self.linear = nn.Linear(inputSize, outputSize)
#super()로 기반 클래스(부모 클래스)를 초기화해줌으로써,
# 기반 클래스의 속성을 subclass가 받아오도록 한다. (초기화를 하지 않으면, 부모 클래스의 속성을 사용할 수 없음)
# 좀 더 명확하게 super를 사용하기 위해서는 단순히 super().__init__()을 하는 것이 아니라 super(파생클래스, self).__init__() 을 해준다.
#이와 같이 적어주면 기능적으로 차이는 없지만, 파생클래스와 self를 넣어서 현재 클래스가 어떤 클래스인지 명확하게 표시해줄 수 있다.
    def forward(self, x):
        # 순전파 연산: 선형 레이어를 통한 입력 데이터 변환
        out = self.linear(x)
        return out
class LinearRegression(nn.Module):
    def __init__(self, inputSize, outputSize):

In [ ]:
inputDim = 1 # 'x'의 차원이 1이 되는 이유를 설명하시오. :

#Linear Regression 모델은 주로 하나의 입력 변수를 기반으로 하여 출력 변수를 예측하는 간단한 선형 모델입니다.
# 입력 변수 'x'가 하나의 차원을 가질 때, 예측 문제는 'x'를 사용하여 'y'를 예측하는 문제가 되며, 이는 하나의 입력 차원을 가지는 것을 의미합니다.
#예를 들어, 주택 가격을 예측하는 경우, 
#주택 크기 (제곱 피트)를 입력으로 사용할 때 'inputDim'은 1이 됩니다.

outputDim = 1 # 'y'의 차원이 1이 되는 이유를 설명하시오. :

#Linear Regression 모델은 연속형 값을 예측하는 회귀 문제를 다룹니다. 
#따라서 예측된 값 'y'의 출력 차원은 1이 됩니다. Linear Regression 모델은 하나의 연속형 값을 예측하므로 'outputDim'은 1이 됩니다. 
#다시 주택 가격 예측 예를 들면, 모델의 출력은 예상 가격으로 하나의 값만을 예측합니다.
learningRate = 0.01
epochs = 100

model = LinearRegression(inputDim, outputDim)

if torch.cuda.is_available():
    model.cuda()

* pytorch에 구현되어있는 loss function, optimizer 사용해보기.

In [ ]:
loss_fn = nn.MSELoss() 
# 주어진 코드에서 loss_fn을 MSE (Mean Squared Error) Loss Function으로 설정하려고 합니다. 
# MSE는 회귀 문제에서 예측값과 실제값 간의 평균 제곱 오차를 계산하는 함수로, 
# 이를 최소화하는 것이 회귀 모델의 목표입니다.
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)
#주어진 코드에서 optimizer를 SGD (Stochastic Gradient Descent) Optimizer로 설정하려고 합니다.
# SGD는 모델의 가중치를 업데이트하는 최적화 알고리즘 중 하나로, 
# 학습률(learning rate)과 같은 하이퍼파라미터를 사용하여 모델 파라미터를 조정합니다.





* 학습과정에서의 이해를 돕기 위해 기능들이 수행하는 역할에 대해 공부하기.

In [ ]:
for epoch in range(epochs):
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(x_train).cuda())
        labels = Variable(torch.from_numpy(y_train).cuda())
    else:
        inputs = Variable(torch.from_numpy(x_train))
        labels = Variable(torch.from_numpy(y_train))

    # optimizer를 zero_grad 시키는 이유: 이전 미니배치의 그래디언트 정보를 초기화하여 현재 미니배치의 그래디언트로 업데이트하기 위함입니다.
    optimizer.zero_grad()

    # 모델을 통해 입력 데이터를 순전파하여 예측값을 얻습니다.
    outputs = model(inputs)

    # 손실 함수를 사용하여 예측값과 실제값 간의 오차(loss)를 계산합니다.
    loss = loss_fn(outputs, labels)
    print(loss)

    # loss function을 backward 시키는 이유: 오차(loss)를 모델 파라미터에 대해 미분하여 그래디언트를 계산하고, 이를 사용하여 모델 파라미터를 업데이트합니다.
    loss.backward()

    # Optimizer를 사용하여 모델 파라미터를 업데이트합니다.
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))


In [ ]:
with torch.no_grad():
    if torch.cuda.is_available():
        predicted = model(Variable(torch.from_numpy(x_train).cuda())).cpu().data.numpy()
    else:
        predicted = model(Variable(torch.from_numpy(x_train))).data.numpy()
    print(predicted)

plt.clf()
plt.plot(x_train, y_train, 'go', label='True data', alpha=0.5)
plt.plot(x_train, predicted, '--', label='Predictions', alpha=0.5)
plt.legend(loc='best')
plt.show()

* Linear regression이 왜 Neural network인지 해당 과제를 예시로 들어 설명하기.

 Linear Regression이 왜 Neural Network의 한 형태로 볼 수 있는지 설명하겠습니다.

Neural Network의 핵심 구성 요소 중 하나는 뉴런(노드)입니다. 각 뉴런은 입력값에 대한 가중치를 가지고, 이 가중치를 사용하여 입력값을 변환하고 활성화 함수를 통과시킵니다. 이런 관점에서 Linear Regression 모델은 Neural Network의 가장 간단한 형태 중 하나입니다.

1. **입력층(Input Layer):** Linear Regression에서 입력 변수(x)는 입력층에 해당합니다. 입력층은 데이터의 특성(feature)을 나타내며, 이 특성에 대한 가중치(weight)를 곱하고 합산하여 출력을 계산합니다.

2. **가중치(Weight):** Linear Regression에서는 가중치가 각 입력 변수와 관련이 있습니다. Neural Network의 뉴런과 마찬가지로 Linear Regression의 입력 변수에 가중치를 곱하여 가중합을 계산합니다.

3. **활성화 함수(Activation Function):** Linear Regression에서는 활성화 함수를 사용하지 않습니다. 그러나 Neural Network의 다른 형태에서는 비선형 활성화 함수(예: 시그모이드, 렐루)를 사용하여 뉴런의 출력을 변환합니다.

4. **출력(Output Layer):** Linear Regression에서 출력은 예측값을 나타냅니다. Neural Network에서도 출력층이 있고, 여기에서 최종 예측이 이루어집니다.

5. **Loss Function과 최적화(Optimization):** Neural Network와 마찬가지로 Linear Regression에서도 Loss Function을 사용하여 예측값과 실제값 간의 오차를 계산하고, 이 오차를 최소화하기 위해 최적화 알고리즘을 사용합니다. 주로 평균 제곱 오차(Mean Squared Error, MSE)를 사용하며, 이를 최소화하기 위해 경사 하강법(Gradient Descent)을 적용합니다.

따라서 Linear Regression은 입력 변수와 가중치를 조합하여 출력을 계산하는 모델로, 이러한 모델은 Neural Network의 가장 간단한 형태 중 하나로 볼 수 있습니다. 뉴런, 가중치, 활성화 함수, 출력층, Loss Function, 최적화와 같은 개념은 Neural Network와 Linear Regression 간에 공통적으로 사용되며, Neural Network는 이러한 개념을 확장하여 더 복잡한 데이터 패턴을 모델링할 수 있습니다.